In [20]:
import nltk
from nltk import  treetransforms
from copy import deepcopy
from cky_implementation import *
# https://parser.kitaev.io/

In [12]:
my_grammar = nltk.data.load("grammar.cfg")  # nltk.CFG.fromstring
print(my_grammar)  #to viz
with open("sentences.txt") as file:
    sentences = file.readlines()
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
    file.close() 

Grammar with 353 productions (start state = S)
    S -> NP VP PUNC
    NP -> NN
    NN -> 'Work'
    VP -> VBZ NP
    VBZ -> 'accelerates'
    NP -> NP PP
    NP -> DT NN
    DT -> 'the'
    NN -> 'growth'
    PP -> IN NP
    IN -> 'of'
    NP -> NNS
    NNS -> 'muscles'
    PUNC -> '.'
    S -> VBZ NP ADJP PUNC
    VBZ -> 'Is'
    NP -> DT NN
    DT -> 'the'
    NN -> 'muscle'
    ADJP -> JJ
    JJ -> 'tired'
    PUNC -> '?'
    S -> RB DT CD PUNC
    RB -> 'Not'
    DT -> 'that'
    CD -> 'one'
    PUNC -> '.'
    S -> NP VP PUNC
    NP -> NNS
    NNS -> 'Muscles'
    VP -> VBP PP
    VBP -> 'grow'
    PP -> IN NP
    IN -> 'in'
    NP -> NP PP
    NP -> NN
    NN -> 'response'
    PP -> IN NP
    IN -> 'to'
    NP -> NN
    NN -> 'work'
    PUNC -> '.'
    S -> S CC S PUNC
    S -> NP VP
    NP -> DT JJ NN NN
    DT -> 'An'
    JJ -> 'average'
    NN -> 'adult'
    NN -> 'male'
    VP -> VBZ VP
    VBZ -> 'is'
    VP -> VBN PRT PP
    VBN -> 'made'
    PRT -> RP
    RP -> 'up'
    P

In [13]:
earley_parser = nltk.parse.EarleyChartParser(my_grammar) 
parsed_sentences = [tuple(earley_parser.parse(sent)) for sent in tokenized_sentences]
bad_indexes = [i for i, tree in enumerate(parsed_sentences) if not bool(tree)]
parsed_sentences = [tree for tree in parsed_sentences if bool(tree)]
print(bad_indexes)
for i in bad_indexes:
    print(" ".join(tokenized_sentences[i]))

[9]
Do you have pain in the muscles on the side of the face ?


In [14]:
good_indexes = tuple(filter(lambda i: i not in bad_indexes, list(range(len(tokenized_sentences)))))
tokenized_sentences = [tokenized_sentences[i] for i in good_indexes]
all_parses = []
for i, sent in enumerate(parsed_sentences):  # 8, 13 and 14 changed to S to run. 9 changed but not running
    parse_count = 0
    for tree in sent:
        print(tree)
        parse_count += 1
    all_parses.append(parse_count)
    print(parse_count)
    print("-------------------------------------")
print(sum(all_parses) / len(parsed_sentences))

(S
  (NP (NN Work))
  (VP
    (VBZ accelerates)
    (NP (NP (DT the) (NN growth)) (PP (IN of) (NP (NNS muscles)))))
  (PUNC .))
1
-------------------------------------
(S
  (NP (NNS Muscles))
  (VP
    (VBP grow)
    (PP (IN in) (NP (NP (NN response)) (PP (TO to) (NP (NN work))))))
  (PUNC .))
(S
  (NP (NNS Muscles))
  (VP
    (VBP grow)
    (PP (IN in) (NP (NP (NN response)) (PP (IN to) (NP (NN work))))))
  (PUNC .))
2
-------------------------------------
(S
  (S
    (NP (DT An) (JJ average) (NN adult) (NN male))
    (VP
      (VBZ is)
      (VP
        (VBN made)
        (PRT (RP up))
        (PP
          (IN of)
          (NP (ADJP (CD 45) (NN %)) (NN skeletal) (NN muscle))))))
  (CC and)
  (S
    (NP (DT an) (JJ average) (NN adult) (NN female))
    (VP
      (VBZ is)
      (VP
        (VBN made)
        (PRT (RP up))
        (PP
          (IN of)
          (NP (ADJP (CD 35) (NN %)) (NN skeletal) (NN muscle))))))
  (PUNC .))
1
-------------------------------------
(S
  (NP (DT The

In [15]:
print(sum(all_parses) / len(parsed_sentences))

2.3684210526315788


In [16]:
my_grammar = nltk.data.load("grammar_cnf.cfg")  
for i, sent in enumerate(tokenized_sentences):  
    cky_table = CKYTable(my_grammar)
    cky_table.table = sent
    print(f"{i+1}. {sent}")
    cky_table.sentenceChecker(sent)
    #cky_table.sentenceChecker("muscles are key for movement .".split())  # testing that sentences are not in the grammar
    print(cky_table.backtrack_lis)
    print()

1. ['Work', 'accelerates', 'the', 'growth', 'of', 'muscles', '.']
Sentence is in the grammar
OrderedDict([(NP, [Det, Nom]), (TOP, [X2, PUNC]), (S, [X2, PUNC]), (X9, [VBZ, NP]), (VP, [VBZ, NP]), (X2, [NP, VP]), (PP, [Prep, NP]), (Nom, [Nom, PP]), (X14, [NN, PP])])

2. ['Muscles', 'grow', 'in', 'response', 'to', 'work', '.']
Sentence is in the grammar
OrderedDict([(TOP, [X2, PUNC]), (S, [X2, PUNC]), (X2, [NP, VP]), (PP, [Prep, NP]), (VP, [VBP, PP]), (Nom, [Nom, PP]), (NP, [Nom, PP]), (X14, [NN, PP])])

3. ['An', 'average', 'adult', 'male', 'is', 'made', 'up', 'of', '45', '%', 'skeletal', 'muscle', 'and', 'an', 'average', 'adult', 'female', 'is', 'made', 'up', 'of', '35', '%', 'skeletal', 'muscle', '.']
Sentence is in the grammar
OrderedDict([(Nom, [Nom, NP]), (NP, [Quant, Nom]), (TOP, [X4, PUNC]), (S, [X4, PUNC]), (ADJP, [Adj, S]), (X12, [VBN, PRT]), (Quant, [Card, NN]), (PP, [Prep, NP]), (VP, [VBZ, VP]), (X2, [NP, VP]), (X15, [Nom, Conj]), (X3, [S, Conj]), (X4, [X3, S])])

4. ['The', 'p

In [18]:
cky_table.table

array([['', {Adj, ADJP}, {Nom, NP}, {TOP, ADJP, S, X2}, {Nom, NP}, '',
        '', {NP, Nom, TOP, X2, ADJP, S}, {TOP, ADJP, S}],
       ['', '', {Nom, NNS, NP}, {TOP, S, X2}, '', '', '', {TOP, S, X2},
        {TOP, S}],
       ['', '', '', {VP, TOP, VBP, S}, {X7}, '', '', {VP, TOP, X7, S},
        ''],
       ['', '', '', '', {Nom, NP, NN}, '', '', {Nom, TOP, NP, S}, ''],
       ['', '', '', '', '', {Det}, '', {NP}, ''],
       ['', '', '', '', '', '', {Adj, ADJP}, {Nom, NP}, ''],
       ['', '', '', '', '', '', '', {Nom, NP, NN}, ''],
       ['', '', '', '', '', '', '', '', {PUNC}],
       ['', '', '', '', '', '', '', '', '']], dtype=object)

In [21]:
for i, sent in enumerate(tokenized_sentences[:1]):  
    cky_table = CKYTable(my_grammar)
    cky_table.table = sent
    print(f"{i+1}. {sent}")
    cky_table.sentenceChecker(sent)
    #cky_table.sentenceChecker("muscles are key for movement .".split())  # testing that sentences are not in the grammar
    print(cky_table.backtrack_lis)
    print()
cky_table.table

1. ['Work', 'accelerates', 'the', 'growth', 'of', 'muscles', '.']
Sentence is in the grammar
OrderedDict([(NP, [Det, Nom]), (TOP, [X2, PUNC]), (S, [X2, PUNC]), (X9, [VBZ, NP]), (VP, [VBZ, NP]), (X2, [NP, VP]), (PP, [Prep, NP]), (Nom, [Nom, PP]), (X14, [NN, PP])])



array([['', {Nom, NP, NN}, '', '', {TOP, S, X2}, '', {TOP, S, X2},
        {TOP, S}],
       ['', '', {VBZ}, '', {VP, TOP, X9, S}, '', {VP, TOP, X9, S}, ''],
       ['', '', '', {Det}, {NP}, '', {NP}, ''],
       ['', '', '', '', {Nom, NP, NN}, '', {Nom, NP, X14}, ''],
       ['', '', '', '', '', {Prep}, {PP}, ''],
       ['', '', '', '', '', '', {Nom, NNS, NP}, ''],
       ['', '', '', '', '', '', '', {PUNC}],
       ['', '', '', '', '', '', '', '']], dtype=object)

In [22]:
cky_table.sentenceChecker("muscles are key for movement .".split())  # testing that sentences are not in the grammar

Sentence is not in the grammar


False

In [23]:
cky_table.sentenceChecker(['Work', 'accelerates', 'the', 'growth', 'of', 'muscles', '.'])  # testing that sentences are not in the grammar

Sentence is in the grammar


True